In [4]:
# =============================================================
# 📘 MODEL TRAINING SCRIPT (v2.2)
# Goal: Predict if NIFTY_50 will go UP (1) or DOWN (0) today
# =============================================================

# 📦 Imports
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# =============================================================
# 🧾 Step 1: Load Processed Dataset
# =============================================================
data_path = "../data/processed/best_features.csv"
df = pd.read_csv(data_path)
print(f"✅ Loaded data with shape: {df.shape}")

# =============================================================
# 🎯 Step 2: Create Target Variable
# =============================================================
df["target"] = (df["NIFTY_50_ret"] > 0).astype(int)

# =============================================================
# 🧹 Step 3: Drop Leakage Columns
# =============================================================
df.drop(columns=["NIFTY_50_ret"], inplace=True, errors="ignore")
if "Date" in df.columns:
    df.drop(columns=["Date"], inplace=True)

print(f"✅ Remaining features: {len(df.columns)-1} + target")

# =============================================================
# 🧩 Step 4: Split into Features (X) and Target (y)
# =============================================================
X = df.drop(columns=["target"])
y = df["target"]

# =============================================================
# 🧪 Step 5: Train–Validation–Test Split (70/15/15)
# =============================================================
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, shuffle=False
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, shuffle=False
)
print(f"✅ Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# =============================================================
# ⚙️ Step 6: Scale Features (KEEP FEATURE NAMES)
# =============================================================
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_val_scaled = pd.DataFrame(
    scaler.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# ✅ SAVE SCALER (NEW)
os.makedirs("../models", exist_ok=True)
joblib.dump(scaler, "../models/feature_scaler.pkl")
print("💾 Scaler saved to ../models/feature_scaler.pkl")

# =============================================================
# 🤖 Step 7: Initialize Models
# =============================================================
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, C=0.1, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=200, learning_rate=0.05, random_state=42),
    "SVM": SVC(kernel="poly", degree=2, probability=True, C=0.5, gamma='scale', coef0=1, class_weight='balanced', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=35, weights='uniform', metric='manhattan'),
    "DecisionTree": DecisionTreeClassifier(max_depth=5, min_samples_split=15, min_samples_leaf=8, class_weight='balanced', random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=200, max_depth=4, min_samples_split=20, min_samples_leaf=10, max_features='sqrt', random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=300, max_depth=7, min_samples_split=10, min_samples_leaf=4, max_features=0.8, random_state=42),
    "GradientBoost": GradientBoostingClassifier(n_estimators=150, learning_rate=0.01, max_depth=3, subsample=0.8, min_samples_split=20, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=3, subsample=0.7, colsample_bytree=0.7, reg_alpha=1.0, reg_lambda=1.0, min_child_weight=5, random_state=42, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(n_estimators=200, learning_rate=0.01, max_depth=3, subsample=0.7, colsample_bytree=0.7, reg_alpha=1.0, reg_lambda=1.0, min_child_samples=20, random_state=42, verbose=-1)
}

# =============================================================
# 🚀 Step 8: Train, Validate, and Save Models
# =============================================================
results = {}
model_save_dir = "../models"

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"🚀 Training {name}...")
    model.fit(X_train_scaled, y_train)

    train_acc = accuracy_score(y_train, model.predict(X_train_scaled))
    val_acc = accuracy_score(y_val, model.predict(X_val_scaled))
    test_acc = accuracy_score(y_test, model.predict(X_test_scaled))
    
    results[name] = {
        "train_acc": train_acc,
        "val_acc": val_acc,
        "test_acc": test_acc
    }
    
    print(f"📊 {name} Accuracies:")
    print(f"   🟩 Train: {train_acc:.4f}")
    print(f"   🟨 Val:   {val_acc:.4f}")
    print(f"   🟥 Test:  {test_acc:.4f}")

# =============================================================
# 🏆 Step 9: Rank Models by Test Accuracy and Save
# =============================================================
sorted_models = sorted(results.items(), key=lambda x: x[1]["test_acc"], reverse=True)

for rank, (name, scores) in enumerate(sorted_models, start=1):
    filename = f"top{rank}_{name}.pkl"
    save_path = os.path.join(model_save_dir, filename)
    joblib.dump(models[name], save_path)
    print(f"💾 Saved {name} as {filename} (Test Acc: {scores['test_acc']:.4f})")

# =============================================================
# 🏁 Step 10: Print Summary
# =============================================================
print("\n" + "="*60)
print("🏁 Model Ranking by Test Accuracy:")
for rank, (name, scores) in enumerate(sorted_models, start=1):
    print(f" {rank}. {name:<15} | Test Acc: {scores['test_acc']:.4f} | Val Acc: {scores['val_acc']:.4f}")
print("="*60)
print("🎯 All models saved by ranking.")


✅ Loaded data with shape: (4295, 15)
✅ Remaining features: 14 + target
✅ Train: (3006, 14), Val: (644, 14), Test: (645, 14)
💾 Scaler saved to ../models/feature_scaler.pkl

🚀 Training LogisticRegression...
📊 LogisticRegression Accuracies:
   🟩 Train: 0.6011
   🟨 Val:   0.6102
   🟥 Test:  0.5643

🚀 Training AdaBoost...
📊 AdaBoost Accuracies:
   🟩 Train: 0.6101
   🟨 Val:   0.6258
   🟥 Test:  0.5736

🚀 Training SVM...
📊 SVM Accuracies:
   🟩 Train: 0.6021
   🟨 Val:   0.6087
   🟥 Test:  0.5628

🚀 Training KNN...
📊 KNN Accuracies:
   🟩 Train: 0.6144
   🟨 Val:   0.6025
   🟥 Test:  0.5535

🚀 Training DecisionTree...
📊 DecisionTree Accuracies:
   🟩 Train: 0.6294
   🟨 Val:   0.6056
   🟥 Test:  0.5612

🚀 Training RandomForest...
📊 RandomForest Accuracies:
   🟩 Train: 0.6287
   🟨 Val:   0.6273
   🟥 Test:  0.5643

🚀 Training ExtraTrees...
📊 ExtraTrees Accuracies:
   🟩 Train: 0.6274
   🟨 Val:   0.6149
   🟥 Test:  0.5643

🚀 Training GradientBoost...
📊 GradientBoost Accuracies:
   🟩 Train: 0.6361
   🟨 